In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import SeparableConv2D, MaxPooling2D, BatchNormalization, Conv2D, Conv1D
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras import models, layers
from keras.models import Model
from keras.layers import BatchNormalization, Activation, Flatten, InputLayer, Lambda, add
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, LearningRateScheduler, CSVLogger

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
# from tensorflow import keras

# from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
'''import tensorflow as tf
config = tf.ConfigProto()
tf.config.gpu.set_per_process_memory_fraction(0.90)
tf.config.gpu.set_per_process_memory_growth(True)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
'''
# Create a session with the above options specified.
# k.tensorflow_backend.set_session(tf.Session(config=config))

'import tensorflow as tf\nconfig = tf.ConfigProto()\ntf.config.gpu.set_per_process_memory_fraction(0.90)\ntf.config.gpu.set_per_process_memory_growth(True)\nconfig = tf.ConfigProto()\nconfig.gpu_options.allow_growth = True\n'

In [0]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 85
l = 6 
num_filter = 35 
compression = 1.0
dropout_rate = 0.20

In [0]:
# Load CIFAR10 Data
from sklearn.model_selection import train_test_split
(temp, y_temp), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_train, X_cv, y_train, y_cv = train_test_split(temp, y_temp, random_state=41, test_size=0.1)

img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 
y_cv = tf.keras.utils.to_categorical(y_cv, num_classes)

170500096/170498071 [==============================] - 2s 0us/step


In [0]:
import numpy as np
X_train = X_train.astype('float64')
X_cv = X_cv.astype('float64')
X_test = X_test.astype('float64')

X_train /= 255
X_cv /= 255
X_test /= 255

print(X_train.shape)
print(X_cv.shape)
print(X_test.shape)

(45000, 32, 32, 3)
(5000, 32, 32, 3)
(10000, 32, 32, 3)


In [0]:
def denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

def transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
        Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    #flat = Flatten()(AvgPooling)
    output = Flatten()(Conv2D(num_classes, 2, activation='softmax')(AvgPooling))
    
    return output

In [0]:
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = transition(First_Block, num_filter, dropout_rate)

Second_Block = denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = transition(Second_Block, num_filter, dropout_rate)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, num_filter, dropout_rate)

Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_142 (Conv2D)             (None, 32, 32, 35)   945         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_141 (BatchN (None, 32, 32, 35)   140         conv2d_142[0][0]                 
__________________________________________________________________________________________________
activation_141 (Activation)     (None, 32, 32, 35)   0           batch_normalization_141[0][0]    
____________________________________________________________________________________________

In [0]:
# determine Loss function and Optimizer


model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

W0905 07:02:16.867691 139990985037696 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:



def decay_fn(epoch, lr):
    if epoch < 50:
        return 0.001
    elif epoch >= 50 and epoch < 75:
        return 0.0001
    else:
        return 0.00001

lr_scheduler = keras.callbacks.LearningRateScheduler(decay_fn)

filepath = "{epoch:03d}-{val_acc:.3f}.hdf5"
model_chkpt = ModelCheckpoint(filepath, monitor = "val_loss", save_best_only=True, verbose = 1)

call = [lr_scheduler, model_chkpt]

In [0]:
model = keras.models.load_model('/content/MODEL5-weights.31-0.96.hdf5')

In [0]:
####### NEW #############
datagen_train = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.125,
    height_shift_range=0.125,
    horizontal_flip=True,
    fill_mode='nearest',
    zoom_range=0.10
)

datagen_train.fit(X_train)

In [0]:
# fits the model on batches with real-time data augmentation:
model.fit_generator(datagen_train.flow(X_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(X_train)//batch_size, epochs=55, initial_epoch=0, verbose=1, validation_data=(X_cv, y_cv), callbacks=call)

Epoch 1/55
703/703 [==============================] - 203s 289ms/step - loss: 1.5496 - acc: 0.4320 - val_loss: 1.7978 - val_acc: 0.4870

Epoch 00001: val_loss improved from inf to 1.79780, saving model to 001-0.487.hdf5
Epoch 2/55
703/703 [==============================] - 183s 261ms/step - loss: 1.1724 - acc: 0.5800 - val_loss: 1.3119 - val_acc: 0.5710

Epoch 00002: val_loss improved from 1.79780 to 1.31185, saving model to 002-0.571.hdf5
Epoch 3/55
703/703 [==============================] - 183s 261ms/step - loss: 0.9937 - acc: 0.6449 - val_loss: 2.1834 - val_acc: 0.4856

Epoch 00003: val_loss did not improve from 1.31185
Epoch 4/55
703/703 [==============================] - 183s 261ms/step - loss: 0.8854 - acc: 0.6842 - val_loss: 1.1471 - val_acc: 0.6600

Epoch 00004: val_loss improved from 1.31185 to 1.14705, saving model to 004-0.660.hdf5
Epoch 5/55
703/703 [==============================] - 183s 260ms/step - loss: 0.8072 - acc: 0.7153 - val_loss: 1.1943 - val_acc: 0.6606

Epoch 0

In [0]:
# fits the model on batches with real-time data augmentation:
model.fit_generator(datagen_train.flow(X_train, y_train, batch_size=128),
                    steps_per_epoch=len(X_train)//128, epochs=65, initial_epoch=62, verbose=1, validation_data=(X_cv, y_cv), callbacks=call)

Epoch 63/65
351/351 [==============================] - 155s 442ms/step - loss: 0.1968 - acc: 0.9300 - val_loss: 0.2938 - val_acc: 0.9110

Epoch 00063: val_loss did not improve from 0.28450
Epoch 64/65
351/351 [==============================] - 155s 442ms/step - loss: 0.1929 - acc: 0.9327 - val_loss: 0.2745 - val_acc: 0.9162

Epoch 00064: val_loss improved from 0.28450 to 0.27454, saving model to 064-0.916.hdf5
Epoch 65/65
351/351 [==============================] - 155s 442ms/step - loss: 0.1894 - acc: 0.9330 - val_loss: 0.2910 - val_acc: 0.9134

Epoch 00065: val_loss did not improve from 0.27454


In [0]:
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 11s 1ms/step
Test loss: 0.3304223409116268
Test accuracy: 0.9038


In [0]:
# Save the trained weights in to .h5 format
model.save("DNST_model2.h5")
print("Saved model to disk")

Saved model to disk
